In [23]:

from pyspark.sql import functions as F

StatementMeta(, 31698c73-9199-4a21-ad7e-ce3d2a00782e, 25, Finished, Available, Finished)

In [1]:
# df_readings = spark.read.table("dbo.silver_cleansed_readings")
# df_transactions = spark.read.table("dbo.silver_cleansed_transactions")
# df_contracts = spark.read.table("dbo.silver_cleansed_contracts")
df_readings = spark.read.table("dbo.iot_silver_cleansed_readings")
df_transactions = spark.read.table("dbo.iot_silver_cleansed_transactions")
df_contracts = spark.read.table("dbo.iot_silver_cleansed_contracts")

StatementMeta(, 65f0ca96-df52-4f34-9fe5-3f53c49474b8, 3, Finished, Available, Finished)

In [2]:
df_readings.printSchema()
df_transactions.printSchema()
df_contracts.printSchema()

StatementMeta(, 65f0ca96-df52-4f34-9fe5-3f53c49474b8, 4, Finished, Available, Finished)

root
 |-- deviceId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- value: double (nullable = true)
 |-- geoLat: double (nullable = true)
 |-- geoLong: double (nullable = true)
 |-- id: string (nullable = true)
 |-- tx_id: string (nullable = true)
 |-- datetime: timestamp (nullable = true)

root
 |-- contractName: string (nullable = true)
 |-- methodName: string (nullable = true)
 |-- type: long (nullable = true)
 |-- status: long (nullable = true)
 |-- cumulativeGasUsed: long (nullable = true)
 |-- txReceipt.logs.removed: boolean (nullable = true)
 |-- transactionHash: string (nullable = true)
 |-- transactionIndex: long (nullable = true)
 |-- blockHash: string (nullable = true)
 |-- blockNumber: long (nullable = true)
 |-- gasUsed: long (nullable = true)
 |-- effectiveGasPrice: long (nullable = true)
 |-- tx_from: string (nullable = true)
 |-- tx_to: string (nullable = true)
 |-- operation: string (nullable = true)
 |-- costEth: double (nullable = true)
 |-- eth_usd

In [26]:
def add_prefix_to_columns(df, prefix):
    return df.select([F.col(c).alias(c if c.startswith(prefix) else f"{prefix}{c}") for c in df.columns])

def remove_prefix_from_columns(df, prefix):
    return df.select([
        F.col(c).alias(c[len(prefix):] if c.startswith(prefix) else c) for c in df.columns])

StatementMeta(, 31698c73-9199-4a21-ad7e-ce3d2a00782e, 28, Finished, Available, Finished)

In [27]:
df_readings = add_prefix_to_columns(df_readings , 'rd_')
df_transactions = add_prefix_to_columns(df_transactions , 'tr_')

StatementMeta(, 31698c73-9199-4a21-ad7e-ce3d2a00782e, 29, Finished, Available, Finished)

In [28]:
df_readings_columns = df_readings.columns
df_transactions_columns = df_transactions.columns


StatementMeta(, 31698c73-9199-4a21-ad7e-ce3d2a00782e, 30, Finished, Available, Finished)

In [29]:
df_transactions.printSchema()

StatementMeta(, 31698c73-9199-4a21-ad7e-ce3d2a00782e, 31, Finished, Available, Finished)

root
 |-- tr_contractName: string (nullable = true)
 |-- tr_methodName: string (nullable = true)
 |-- tr_type: long (nullable = true)
 |-- tr_status: long (nullable = true)
 |-- tr_cumulativeGasUsed: long (nullable = true)
 |-- tr_transactionHash: string (nullable = true)
 |-- tr_transactionIndex: long (nullable = true)
 |-- tr_blockHash: string (nullable = true)
 |-- tr_blockNumber: long (nullable = true)
 |-- tr_gasUsed: long (nullable = true)
 |-- tr_effectiveGasPrice: long (nullable = true)
 |-- tr_tx_from: string (nullable = true)
 |-- tr_tx_to: string (nullable = true)
 |-- tr_operation: string (nullable = true)
 |-- tr_costEth: double (nullable = true)
 |-- tr_eth_usd_rate: double (nullable = true)
 |-- tr_costUSD: double (nullable = true)
 |-- tr_datetime: timestamp (nullable = true)



In [30]:
df_readings_transactions = df_readings.join(df_transactions , df_readings.rd_tx_id == df_transactions.tr_transactionHash , how="inner")
df_readings_transactions.printSchema()

StatementMeta(, 31698c73-9199-4a21-ad7e-ce3d2a00782e, 32, Finished, Available, Finished)

root
 |-- rd_deviceId: string (nullable = true)
 |-- rd_type: string (nullable = true)
 |-- rd_value: double (nullable = true)
 |-- rd_geoLat: double (nullable = true)
 |-- rd_geoLong: double (nullable = true)
 |-- rd_id: string (nullable = true)
 |-- rd_tx_id: string (nullable = true)
 |-- rd_datetime: timestamp (nullable = true)
 |-- tr_contractName: string (nullable = true)
 |-- tr_methodName: string (nullable = true)
 |-- tr_type: long (nullable = true)
 |-- tr_status: long (nullable = true)
 |-- tr_cumulativeGasUsed: long (nullable = true)
 |-- tr_transactionHash: string (nullable = true)
 |-- tr_transactionIndex: long (nullable = true)
 |-- tr_blockHash: string (nullable = true)
 |-- tr_blockNumber: long (nullable = true)
 |-- tr_gasUsed: long (nullable = true)
 |-- tr_effectiveGasPrice: long (nullable = true)
 |-- tr_tx_from: string (nullable = true)
 |-- tr_tx_to: string (nullable = true)
 |-- tr_operation: string (nullable = true)
 |-- tr_costEth: double (nullable = true)
 |--

In [31]:
df_valid_readings_transactions = df_readings_transactions.filter(F.col("rd_datetime") > "2025-01-01")
df_invalid_readings_transactions = df_readings_transactions.filter(F.col("rd_datetime") < "2025-01-01")
print('Valid transactions: ',df_valid_readings_transactions.count())
print('Invalid transactions: ',df_invalid_readings_transactions.count())

StatementMeta(, 31698c73-9199-4a21-ad7e-ce3d2a00782e, 33, Finished, Available, Finished)

Valid transactions:  3393
Invalid transactions:  1273


In [32]:
df_curated_readings = df_valid_readings_transactions.select(df_readings_columns)
df_curated_transactions = df_valid_readings_transactions.select(df_transactions_columns)
df_curated_contracts = df_contracts

df_curated_readings.printSchema()
df_curated_transactions.printSchema()
df_contracts.printSchema()

StatementMeta(, 31698c73-9199-4a21-ad7e-ce3d2a00782e, 34, Finished, Available, Finished)

root
 |-- rd_deviceId: string (nullable = true)
 |-- rd_type: string (nullable = true)
 |-- rd_value: double (nullable = true)
 |-- rd_geoLat: double (nullable = true)
 |-- rd_geoLong: double (nullable = true)
 |-- rd_id: string (nullable = true)
 |-- rd_tx_id: string (nullable = true)
 |-- rd_datetime: timestamp (nullable = true)

root
 |-- tr_contractName: string (nullable = true)
 |-- tr_methodName: string (nullable = true)
 |-- tr_type: long (nullable = true)
 |-- tr_status: long (nullable = true)
 |-- tr_cumulativeGasUsed: long (nullable = true)
 |-- tr_transactionHash: string (nullable = true)
 |-- tr_transactionIndex: long (nullable = true)
 |-- tr_blockHash: string (nullable = true)
 |-- tr_blockNumber: long (nullable = true)
 |-- tr_gasUsed: long (nullable = true)
 |-- tr_effectiveGasPrice: long (nullable = true)
 |-- tr_tx_from: string (nullable = true)
 |-- tr_tx_to: string (nullable = true)
 |-- tr_operation: string (nullable = true)
 |-- tr_costEth: double (nullable = true

In [11]:
#
# preparing devices by zone
#

StatementMeta(, 31698c73-9199-4a21-ad7e-ce3d2a00782e, 13, Finished, Available, Finished)

In [33]:
df_curated_readings = remove_prefix_from_columns(df_curated_readings , 'rd_')
df_curated_transactions = remove_prefix_from_columns(df_curated_transactions , 'tr_')

df_curated_readings.write.format("delta").mode("overwrite").saveAsTable("iot_gold.readings")
df_curated_transactions.write.format("delta").mode("overwrite").saveAsTable("iot_gold.transactions")
df_curated_contracts.write.format("delta").mode("overwrite").saveAsTable("iot_gold.contracts")


StatementMeta(, 31698c73-9199-4a21-ad7e-ce3d2a00782e, 35, Finished, Available, Finished)